In [7]:
#Audio into Spectrogram
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import librosa.display

In [22]:
import pylab
save_path = 'spectrograms/MJF/Spec1.png'
#pylab.axis('off') # no axis
pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
#S = librosa.feature.melspectrogram(y=sig, sr=fs)
librosa.display.specshow(librosa.amplitude_to_db(S_foreground[:, idx], ref=np.max),
                          sr=sr)

pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
pylab.close()


In [39]:
wavfile = []
for root, dirs, files in os.walk('Letterman'):
    wavfile += glob.glob(os.path.join(root,'*.wav'))
print(wavfile)

['Letterman/hKzUVPnT40o6.wav', 'Letterman/DVK2GgfGMEY19.wav', 'Letterman/DVK2GgfGMEY31.wav', 'Letterman/DVK2GgfGMEY25.wav', 'Letterman/hKzUVPnT40o22.wav', 'Letterman/DVK2GgfGMEY24.wav', 'Letterman/DVK2GgfGMEY30.wav', 'Letterman/DVK2GgfGMEY18.wav', 'Letterman/hKzUVPnT40o7.wav', 'Letterman/hKzUVPnT40o5.wav', 'Letterman/DVK2GgfGMEY26.wav', 'Letterman/DVK2GgfGMEY32.wav', 'Letterman/hKzUVPnT40o20.wav', 'Letterman/3XoIDGs11kE5.wav', 'Letterman/3XoIDGs11kE4.wav', 'Letterman/hKzUVPnT40o21.wav', 'Letterman/DVK2GgfGMEY33.wav', 'Letterman/DVK2GgfGMEY27.wav', 'Letterman/hKzUVPnT40o4.wav', 'Letterman/DVK2GgfGMEY23.wav', 'Letterman/DVK2GgfGMEY37.wav', 'Letterman/hKzUVPnT40o19.wav', 'Letterman/3XoIDGs11kE0.wav', 'Letterman/3XoIDGs11kE1.wav', 'Letterman/hKzUVPnT40o18.wav', 'Letterman/DVK2GgfGMEY36.wav', 'Letterman/DVK2GgfGMEY22.wav', 'Letterman/hKzUVPnT40o3.wav', 'Letterman/DVK2GgfGMEY34.wav', 'Letterman/DVK2GgfGMEY20.wav', 'Letterman/3XoIDGs11kE3.wav', 'Letterman/3XoIDGs11kE2.wav', 'Letterman/DVK2Ggf

In [40]:
for i in wavfile:
    
    y, sr = librosa.load(i)
    # And compute the spectrogram magnitude and phase
    S_full, phase = librosa.magphase(librosa.stft(y))

    idx = slice(*librosa.time_to_frames([0, 10], sr=sr))
    #filter out background noise to 'clean' the audio file to have just voices
    S_filter = librosa.decompose.nn_filter(S_full,
                                           aggregate=np.median,
                                           metric='cosine',
                                           width=int(librosa.time_to_frames(2, sr=sr)))

    # The output of the filter shouldn't be greater than the input
    # if we assume signals are additive.  Taking the pointwise minimium
    # with the input spectrum forces this.
    S_filter = np.minimum(S_full, S_filter)
    # We can also use a margin to reduce bleed between the vocals and instrumentation masks.
    # Note: the margins need not be equal for foreground and background separation
    margin_i, margin_v = 2, 10
    power = 2

    mask_i = librosa.util.softmask(S_filter,
                                   margin_i * (S_full - S_filter),
                                   power=power)

    mask_v = librosa.util.softmask(S_full - S_filter,
                                   margin_v * S_filter,
                                   power=power)


    S_foreground = mask_v * S_full
    S_background = mask_i * S_full
    save_path = 'spectrograms/'+i[:-3]+'png'
    #pylab.axis('off') # no axis
    pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[]) # Remove the white edge
    #S = librosa.feature.melspectrogram(y=sig, sr=fs)
    librosa.display.specshow(librosa.amplitude_to_db(S_foreground[:, idx], ref=np.max),
                              sr=sr)

    pylab.savefig(save_path, bbox_inches=None, pad_inches=0)
    pylab.close()

    